# Język Python - Wykład 6.

"The story of Jython begins one summer in Ashland, Oregon. I was juggling in a park behind a theater when I met Pavel Curtis, a scientist at Xerox PARC, who wanted to pass clubs. While we were juggling together he told me about a wonderful new programming language called Python. **Writing code in Python felt like writing the sort of natural informal code that developers would use when they wanted to quickly share ideas. It was executable pseudo-code.**"
Jim Hugunin - http://hugunin.net/story_of_jython.html

## Dekoratory

In [ ]:
def say_even_more(*args, **kwargs):
    print(args)  # krotka
    print(kwargs)  # słownik

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

def logged(f):
    def logged_f(*args, **kwargs):
        logging.debug("Called {!r} with params {} and {}".format(f.__name__, args, kwargs))
        ret_val = f(*args, **kwargs)
        logging.debug("{!r} returned {!r}".format(f.__name__, ret_val))
        return ret_val
    return logged_f

In [ ]:
say_even_more = logged(say_even_more)

In [ ]:
say_even_more(1, 2)

In [ ]:
@logged
def g(n):
    print("Another simple method printing {}.".format(n))

In [ ]:
g(2)

In [ ]:
def synchronized(f):
    return f

In [ ]:
# Example ...
@synchronized
@logged
def myfunc(arg1, arg2, *args):
    # ...do something
    pass

In [ ]:
def entry_exit(f):
    def new_f():
        print("Entering", f.__name__)
        f()
        print("Exited", f.__name__)
    return new_f

In [ ]:
@entry_exit
def func1():
    print("inside func1()")
func1()

In [ ]:
class EntryExit:
    
    def __init__(self, f):
        self.f = f
        self.n = 0
    
    def __call__(self):
        self.n += 1
        print("Entering", self.f.__name__, self.n, "time" + ("s" if self.n > 1 else ""))
        self.f()
        print("Exited", self.f.__name__)

        
@EntryExit
def func1():
    print("inside func1()")

In [ ]:
type(func1)

In [ ]:
func1()

In [ ]:
from random import randint

@moving_avg(4)
def foo():
    return randint(1, 100)

In [ ]:
def moving_avg(

In [ ]:
class Decorator:
    
    def __init__(self, arg):
        self.arg = arg
    
    def __call__(self, cls):
        class Wrapped(cls):
            classattr = self.arg
            def new_method(self, value):
                return value * 2
        return Wrapped

In [ ]:
@Decorator("decorated class")
class TestClass:
    
    def new_method(self, value):
        return value * 3
    
t = TestClass()

print(t.new_method(5))

In [ ]:
t.classattr

In [ ]:
def my_decorator(f):
    def wrapper(*args, **kwargs):
        print('Calling decorated function')
        return f(*args, **kwargs)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

In [ ]:
example()

In [ ]:
print(example.__doc__)

In [ ]:
print(example.__name__)

In [ ]:
def my_decorator(f):
    def wrapper(*args, **kwargs):
        print('Calling decorated function')    
        return f(*args, **kwargs)
    wrapper.__doc__ = f.__doc__
    wrapper.__name__ = f.__name__
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

In [ ]:
example()
print(example.__doc__)
print(example.__name__)

Bazinga!

In [ ]:
from functools import wraps

def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

In [ ]:
print(example.__doc__)
print(example.__name__)

* przy pisaniu dekoratorów **powinniśmy** dekorować funkcję dekorującą dekoratorem *wraps* z funkcją dekorowaną jako argument
* przypiszemy nowej funkcji wszystkie potrzebne atrybuty starej funkcji, żeby mogła ją całkowicie podmienić 

## Pliki

In [ ]:
from pprint import pprint

with open('Lecture4.ipynb') as notebook:
    lines = notebook.readlines()
    pprint(lines)

In [ ]:
with open('Lecture4.ipynb') as notebook:
    for line in notebook:
        pprint(line)
        break

Czy pliki __zawsze__ otwieramy w bloku `with`?

## Type Hints

https://docs.python.org/3/library/typing.html

PEP 483, 484, 526, 544, 586, 589 i 591

Python 3.6+

In [ ]:
a: "hello" = 1
print(a)
print(__annotations__)

In [ ]:
def foo(a: "in") -> "out":
    pass

foo.__annotations__

In [ ]:
from typing import List, Dict, NewType, Any

Vector = List[float]
# 3.9+: Vector = list[float]

In [ ]:
a: Vector = []

In [ ]:
b: int
b

In [ ]:
c: int
c: float

In [ ]:
def mean(d: Dict[Any, float]) -> float:
    return sum(d.values())/len(d)

In [ ]:
ID = NewType('ID', int)

user_id: ID = ID(14)

Duck types:
- Mapping
- MutableMapping
- Sequence
- Iterable
- Callable (Callable[..., float])

- Optional
- Any
- Union
- None
- TypeVar

## Moduły i pakiety

### Wyrażenie import

* Zmienna `__name__` zawiera nazwę bieżącego modułu

    #!/usr/bin/python3
    # Filename: using_name.py
    
    if __name__ == '__main__':
        print('This program is being run by itself')
    else:
        print('I am module {!r} imported from another module'.format(__name__))

Przykładowe uruchomienia:
    
    $ python3 using_name.py
    This program is being run by itself
    
    $ python3
    >>> import using_name
    I am module 'using_name' imported from another module
    >>>

* **import**
    * dołącza plik z $PYTHONPATH: .:/usr/local/lib/python
    
* **import test**
    * Do x w test odwołujemy się przez "test.x"
    
* **from test import x**
    * Do x w test odwołujemy się przez "x"
    * Nie mamy dostępu do innych składowych, ani do samego "test"
    
* **import test.x.y** 
    * Do y odwołujemy się przez "test.x.y"    
* **from test.x import y**
    * Do y odwołujemy się przez "y"
* **from test import ***
    * Wczytuje wszystko z test. Do x w test odwołujemy się przez "x" (nie zalecane)
    
* **import test as theTest** 
    * Do x w test odwołujemy się przez "theTest.x"
* **from test import x as my_x**
    * Do x odwołujemy się przez "my_x" 
   


* Symbol zaimportowany zostaje dołączony do przestrzeni nazw
* Importowanie modułu (dowolnym sposobem) wykonuje **cały** jego kod
* Kolekcja modułów (pakiet) znajduje się w katalogu
* Pakiet musi zawierać plik \__init\__.py
* Pakiet może zawierać podpakiety

* **from .other_module import func** lub **from ..other_module import func**
    * do używania wewnątrz pakietu do importowania innych modułów z tego samego pakietu

    sound/                   Top-level package
      __init__.py            Initialize the sound package
      formats/               Subpackage for format conversions
          __init__.py
          wavread.py
          wavwrite.py
          aiffread.py
          aiffwrite.py
          auread.py
          auwrite.py

    >>> from sound.formats import wavwrite

### dir()

In [ ]:
import sys
dir(sys) # get list of attributes for sys module

In [ ]:
%sx pwd

In [ ]:
%sx ls

In [ ]:
%sx mkdir -p agh

In [ ]:
%cd agh

In [ ]:
%sx pwd

In [ ]:
%%writefile __init__.py
print("I am init")

In [ ]:
%%writefile student.py

print("Importing student")

def _opinion():
    return " rocks!"

def scream():
    print("AGH" + _opinion())
    
print("Name:", __name__)

In [ ]:
%cd ..

In [ ]:
from agh import student
student.scream()

In [ ]:
from agh.student import scream
scream()

In [ ]:
from agh import student.scream

In [ ]:
import agh.student

agh.student.scream()

In [ ]:
%run agh/student.py

In [ ]:
student.scream()

In [ ]:
student._opinion()

In [ ]:
dir()

In [ ]:
%reset

In [ ]:
dir()

## Kilka słów o logicznych typach danych

Ile ruchów trzeba wykonać, żeby włożyć słonia do lodówki?

Do umieszczenia w repozytorium przed zwolnieniem się z firmy (za https://gist.github.com/aras-p/6224951)
     
     #define true ((rand()&15)!=15)
     
     #define if(x) if ((x) && (rand() < RAND_MAX * 0.99))

Słowa kluczowe - traktowane specjalnie przez parser

In [ ]:
from keyword import kwlist

print(kwlist)

Nie można użyć słów kluczowych do nazywania zmiennych i funkcji

In [ ]:
for = 8
while = "aaa"

Wbudowane - typy, funkcje, wyjątki. Traktowane przez parser jak identyfikatory tworzone przez programistę. Można (o zgrozo) używać ich w przypisaniach.

In [ ]:
print(dir(__builtins__))

In [ ]:
int = "ojej"
len = lambda x : 137

In [ ]:
print(len([0, 1, 2, 3]))
'True' in dir(__builtins__)

* Porównania

  * Python2 (od 2.3): wbudowany typ `bool`, dwie wartości ``True`` i ``False`` **nie będące słowami kluczowymi**
  * Python3: wbudowana klasa `bool`, dwie możliwe wartości będące obiektami tej klasy: ``True`` i ``False`` - **słowa kluczowe**
  
  http://python-history.blogspot.com/2013/11/story-of-none-true-false.html

In [ ]:
%%python3
True = 0

In [ ]:
%%python2
True = 0

if True: print("OK")

In [ ]:
%%python2
def guess(number=23):
    running = True

    while running:
        guess = int(raw_input('Enter an integer : '))

        if guess == number:
            print 'Congratulations, you guessed it.'
            running = False # this causes the while loop to stop
        elif guess < number:
            print('No, it is a little higher than that.')
        else:
            print('No, it is a little lower than that.')
    else:
        print('The while loop is over.')
        # Do anything else you want to do here
    print('Done')

True = 0
guess()

In [ ]:
%reset

Porównania:

* nie używać "``== True``" ani "`is True`" do sprawdzania prawdziwości

     * Dobrze: ``if greeting:``
     * Źle:    ``if greeting == True:``
     * Jeszcze gorzej: ``if greeting is True:``

## Programowanie funkcyjne

* filter - filtrowanie kolekcji
* map - modyfikacja kolekcji
* reduce - redukowanie kolekcji (wyliczanie wartości z jej elementów), w Python3 w module functools

In [ ]:
l = list(range(10))
print(list(filter(lambda x: not x%2, l)))
print([x for x in l if not x%2])

In [ ]:
print(list(map(lambda x: x**2, l)))
print([x**2 for x in l])

Zamiast reduce:
- sum
- set.union
- set.intersection

In [ ]:
l = [[x, x+1] for x in range(4)]
print(l)
sum(l, start=[])

Policzyć, ile owoców wspólnie mają Merry i Tom:

In [ ]:
fruits = {
    'Merry': [
        ('apple', 5),
        ('orange', 3),
    ],
    'John': [
        ('berries', 1),
        ('orange', 10),
    ],
    'Tom' : [
        ('peach', 2),
    ],
}
list(fruits.items())

Filtrujemy "bazę" z owoców Johna:

In [ ]:
f = filter(lambda x: x[0] in ['Merry', 'Tom'], fruits.items())
list(f)

Usuwamy imiona, nie są nam już potrzebne:

In [ ]:
mapa1 = map(lambda y: y[1], 
        filter(lambda x: x[0] in ['Merry', 'Tom'], fruits.items())
)
list(mapa1)

Alternatywnie

In [ ]:
[val for name, val in fruits.items() if name in {'Merry','Tom'}]

Sklejamy listę list w jedną listę:

In [ ]:
from itertools import chain
list(chain(
    *map(lambda y: y[1], 
        filter(lambda x: x[0] in ['Merry', 'Tom'], fruits.items())
    )
))

Mapujemy listę tupli na listę liczb

In [ ]:
mapa2 = map(lambda f: f[1], 
    chain(
        *map(lambda y: y[1], 
            filter(lambda x: x[0] in ['Merry', 'Tom'], fruits.items())
        )
    )
)
list(mapa2)

Finalnie redukujemy listę do sumy elementów:

In [ ]:
sum( 
    map(lambda f: f[1], 
        chain(
            *map(lambda y: y[1], 
                filter(lambda x: x[0] in ['Merry', 'Tom'], fruits.items())
            )
        )
    )
)

Możemy także użyć gotowego operatora dodawania z modułu operator:

In [ ]:
import operator
import functools
functools.reduce(lambda a,b: a+b, 
    map(lambda f: f[1], 
        chain(
            *map(lambda y: y[1], 
                filter(lambda x: x[0] in ['Merry', 'Tom'], fruits.items())
            )
        )
    )
)

In [ ]:
list(chain(*[item[1] for item in fruits.items() if item[0] in {'Merry','Tom'}]))

In [ ]:
sum(it[1] for it in chain(*[item[1] for item in fruits.items() if item[0] in {'Merry','Tom'}]))

In [ ]:
fruits = {
    'Merry': [
        ('apple', 5),
        ('orange', 3),
    ],
    'John': [
        ('berries', 1),
        ('orange', 10),
    ],
    'Tom' : [
        ('peaches', 2),
    ],
}
fruits

In [ ]:
sum([y[1] for x in [owoce for name, owoce in fruits.items() if name in ["Merry", "Tom"]] for y in x])